In [ ]:
import warnings

warnings.filterwarnings("ignore")

# Equi7Grid tiling system

The core class in the Equi7Grid is the `Equi7TilingSystem` and represents an Equi7Grid continental subgrid as a regular, multi-level (e.g., zoom, tiling level, ...) projected tiling system. A projected tiling system is defined as follows:

- a projection represented via a CRS definition (EPSG, PROJ4, WKT, …) and a projection zone defining the validity of coordinates
- a regular tiling scheme slicing the projection extent into smaller regular tiles with a given tile and pixel size (= sampling); tiling schemes are stacked in a tiling system to form a hierarchy of tiling levels (generic realisation of [OGC's Tile Matrix Set](https://docs.ogc.org/is/17-083r4/17-083r4.html#toc14)). Equi7Grid's standard tiling system consists of three tiling levels: T1 (100km tile size), T3 (300km tile size), and T6 (600km tile size).

On a code-level, `Equi7TilingSystem` inherits from `pytileproj`'s [`RegularProjTilingSystem`](https://tuw-geo.github.io/pytileproj/latest/guides/tiling_system.html#regular-projected-tiling-system) and extends it with a definition of land masses for each continent to decide if a tile is covering land or not.

Lets dive deeper into how an `Equi7TilingSystem` instance can be used. As an example, we assume we retrieved the instance for the European Equi7Grid tiling system ("EU").

In [ ]:
from equi7grid import get_standard_equi7grid

e7grid = get_standard_equi7grid({"T6": 500, "T3": 500})
e7eu = e7grid.EU

```python
e7af = ...
```

### Projection system interactions

Often it is required to transform coordinates back and forth between different projection systems. `Equi7TilingSystem` provides several methods to do this in a straighforward way. If you want transform geographic coordinates to projected coordinates, you can use `lonlat_to_xy`:

In [ ]:
lon, lat = 16.3926, 48.1674
proj_coord = e7eu.lonlat_to_xy(lon, lat)
proj_coord

The same can be done the other way around:

In [ ]:
e7eu.xy_to_lonlat(proj_coord.x, proj_coord.y)

Note that for coordinates outside the projection zone, an error will be raised:

In [ ]:
lon, lat = -100, 30
e7eu.lonlat_to_xy(lon, lat)

You can check in advance if a point or geometry is within the projection by using Python's `in` operator:

In [ ]:
from pytileproj import GeogCoord

coord = GeogCoord(lon, lat)
coord in e7eu  # ty: ignore[unsupported-operator]

As you have seen in this example `pytileproj` offers several classes to store geometries along with projection information: `ProjCoord` and `ProjGeom` as the core classes and `GeogCoord` and `GeogGeom` to represent geometries in the geographic "LatLon" system (more details can be found [here](https://tuw-geo.github.io/pytileproj/latest/guides/geometry.html)).

If you want to know the units of the projection system, you can retrieve this information via:

In [ ]:
e7eu.unit

### Tiles

Equi7 tiling systems create tiles on the fly, since their regular tiling scheme allows an efficient computation of tile properties. A tile can be created by providing a location of interest with the following methods: `get_tile_from_lonlat`, `get_tile_from_xy`, and `get_tile_from_coord`. Here is an example: 

In [ ]:
lon, lat = 16.3926, 48.1674
e7eu.get_tile_from_lonlat(lon, lat, tiling_id="T6")

If you are interested in retrieving a list of tiles intersecting with a region of interest in geographic coordinates, you can use `get_tiles_in_geog_bbox`. It returns a generator to enable lazy tile retrieval.

In [ ]:
list(e7eu.get_tiles_in_geog_bbox((16, 48, 18, 50), tiling_id="T6"))

Similar queries can be done with native coordinates 

In [ ]:
list(e7eu.get_tiles_in_bbox((5155982, 1505898, 6323863, 2140396), tiling_id="T6"))

or by using a geometry.

In [ ]:
from pytileproj import GeogGeom
from shapely import Polygon

geom = Polygon([(16, 48), (18, 48), (18, 50), (16, 50)])
ggeom = GeogGeom(geom=geom)
list(e7eu.get_tiles_in_geom(ggeom, tiling_id="T3"))

Since tiling systems represent an hierarchy of different tilings, `Equi7TilingSystem` also allows to trace parent and children tiles. You can obtain the parent of a tile with

In [ ]:
e7eu.get_parent_from_name("EU_E048N012T3")

and the children with

In [ ]:
list(e7eu.get_children_from_name("EU_E054N016T6"))

### Export

A tiling system offers several export methods, e.g., an export to a `GeoDataFrame` (`to_geodataframe`) or a shapefile (`to_shapefile`). Here is an example how to generate a `GeoDataFrame`:

In [ ]:
e7eu.to_geodataframe()

If you want to share an OGC compliant definition of the tiling system, you can use `to_ogc_standard` or `to_ogc_json`:

In [ ]:
import json
import pprint
from pathlib import Path

ogc_ts_path = Path("my_equi7_system.json")
e7eu.to_ogc_json(ogc_ts_path)

with ogc_ts_path.open() as f:
    ogc_ts = json.load(f)

pprint.pprint(ogc_ts)  # noqa: T203

In [ ]:
ogc_ts_path.unlink(missing_ok=True)

### Visualisation

A Equi7 tiling systems can be also visualised on a map in a similar manner as a `Equi7Tile` object, if the optional dependencies `matplotlib` and `cartopy` are installed.

In [ ]:
import matplotlib.pyplot as plt

plt.figure(figsize=(20, 28))
e7eu.plot(
    tiling_id="T3",
    label_tile=True,
    label_size=5,
    plot_zone=True,
    facecolor="#2cee768f",
    alpha=0.6,
    extent=[-6e5, -6e5, 9e6, 6e6],
)